# Technical challenge | Cristiano Martins Monteiro

Importing the libraries

In [41]:
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, LpBinary, get_solver, LpStatusOptimal
from pathlib import Path
import os.path

Defining the classes

In [42]:
# This Item class is a super-class for Cylinder, Box and Container. It defines the instances' identifier and optimization variables.
class Item:
    def __init__(self, identifier):
        self.identifier = identifier

        # In this proposed formulation, the boxes do not need a variable.
        if not isinstance(self, Box):
            self.variable = LpVariable(name=identifier, cat=LpBinary)

    # Converts a list of Item's variables to a Numpy array and sums it.
    @staticmethod
    def sum_variables(list_of_items):
        numpy_array = np.asarray([item.variable for item in list_of_items])
        return np.sum(numpy_array)
    
    # Builds a Numpy array of cylinder variables multiplying each cylinder's weight and volume.
    @staticmethod
    def volume_weight_times_cylinders_vars(list_of_cylinders):
        # Builds an array with two columns: the first one multiplying the volumes to the variables,
        #                                   and the second one multiplying the weights to the variables.
        numpy_array = np.asarray([( cylinder.volume * cylinder.variable,
                                    cylinder.weight * cylinder.variable) for cylinder in list_of_cylinders])

        # Sum the arrays according with the columns (axis = 1). This sum will return one value for each column (volume and weight).
        result = np.sum(numpy_array, axis=0)
        return result[0], result[1]

# Defines the cylinders.
class Cylinder(Item):
    def __init__(self, identifier, weight, volume, density, box):
        super().__init__(identifier)

        self.weight = weight
        self.volume = volume
        self.density = density

        # Each cylinder is inside a box. Both cylinder and box has a reference to each other.
        self.box = box
        self.box.cylinders.append(self)

# Defines the boxes.
class Box(Item):
    def __init__(self, identifier, container):
        super().__init__(identifier)

        # Maintains the box's cylinders.
        self.cylinders = list()

        # Each box is inside a container. Both box and container has a reference to each other.
        self.container = container
        self.container.boxes.append(self)

# Defines the containers.
class Container(Item):
    def __init__(self, identifier):
        super().__init__(identifier)

        # Maintains the container's boxes.
        self.boxes = list()

Reading the data and constructing the instances

In [43]:
# Loading a data frame with only the columns and rows with data.
data_frame = pd.read_excel('Avaliacao_Otimizacao.xlsx', usecols='C:H', skiprows=18)

# Defining dictionaries to store all items. The key for each item is its identifier attribute.
containers = dict()
boxes = dict()
cylinders = dict()
# Iterating over the data
for index, row in data_frame.iterrows():
    # Setting the identifier of each item
    container_id = row['Container']
    box_id = container_id + '|' + row['Box']
    cylinder_id = box_id + '|' + str(row['Cylinder'])

    # Checking if it is the first time this container appears. If not, this container is constructed and stored in the dictionary.
    if container_id not in containers:
        container = Container(identifier=container_id)
        containers[container_id] = container
    # Accessing the already constructed container.
    else:
        container = containers[container_id]
    
    # Checking and doing the same, but for the box.
    if box_id not in boxes:
        box = Box(identifier=box_id, container=container)
        boxes[box_id] = box
    else:
        box = boxes[box_id]
    
    # Checking is not required for cylinders because every line in the data frame is a new cylinder.
    cylinder = Cylinder(identifier=cylinder_id,
                        weight=row['Cylinder weight (g)'],
                        volume=row['Cylinder volume (mL)'],
                        density=row['Density (g/mL)'],
                        box=box)
    cylinders[cylinder_id] = cylinder

# Defining the cylinders' identifier as an index for the data_frame rows. Each cylinder's identifier also contains the box and container identifiers.
data_frame['identifier'] = [cylinder.identifier for cylinder in cylinders.values()]
data_frame = data_frame.set_index('identifier')

Building the optimization model

In [44]:
# Defining the optimization model object.
problem = LpProblem(name="Raizen_Challenge")

# Defining the objective function. For this problem, any feasible solution is good enough.
sum_all_containers_vars = Item.sum_variables(containers.values())
sum_all_cylinders_vars = Item.sum_variables(cylinders.values())
problem += 0 * sum_all_containers_vars + 0 * sum_all_cylinders_vars, "Sum of Zeros. Any feasible solution is good enough."

# Defining the constraints. The constraints are defined in the same order they are described in the challenge.
TOTAL_CONTAINERS = 35
problem += sum_all_containers_vars == TOTAL_CONTAINERS, "Choosing exactly 35 containers."

sum_volume_cylinders_vars, sum_weight_cylinders_vars = Item.volume_weight_times_cylinders_vars(cylinders.values())
TOTAL_VOLUME = 5163.69
problem += sum_volume_cylinders_vars == TOTAL_VOLUME, "The selected cylinders' volume must sum to this value."

TOTAL_WEIGHT = 18844
problem += sum_weight_cylinders_vars == TOTAL_WEIGHT, "The selected cylinders together must weigh this much."

for container in containers.values():
    for box in container.boxes:
        sum_cylinders_of_box_vars = Item.sum_variables(box.cylinders)
        # If a container is selected, at least one cylinder from any of its boxes must be selected as well.
        # The len(box.cylinders) works as a Big M. Two constraints were needed to tangle the containers to the boxes.
        problem += container.variable <= sum_cylinders_of_box_vars, "When a container is chosen, at least one of its boxes must be used " + box.identifier + "."
        problem += len(box.cylinders) * container.variable >= sum_cylinders_of_box_vars, "When a box is used, its container must be chosen " + box.identifier + "."
        problem += sum_cylinders_of_box_vars <= 1, "Choosing at maximum one cylinder per box " + box.identifier + "."

Solving the MIP

In [45]:
# Defining the solver and setting it to no print log messages
solver_object = get_solver(solver='COIN_CMD', msg=0)
pulp_status_solve = problem.solve(solver_object)

Defining a function for accessing the optimal solution and checking if it is correct

In [46]:
# All different optimal solutions will be stored in this set. This set will be used to check if all generated optimal solutions are indeed unique.
all_optimal_solutions = set()

# Access the current optimal solution. If it is correct, returns the chosen cylinders identifiers and objects in lists. Otherwise, returns None.
def evaluate_solution(problem):
    used_containers = set()
    sum_volumes = 0
    sum_weights = 0
    used_boxes = set()
    chosen_cylinders_ids = list()
    chosen_cylinders = list()
    for variable in problem.variables():
        if variable.varValue == 1:
            # It is possible to differentiate containers from cylinders by the number of pipe symbols in the variable's name.
            # Container variables have smaller names, with no pipe symbol separating them from the box and cylinder identifiers.
            split_variable_name = variable.name.split('|')
            if len(split_variable_name) == 1:
                used_containers.add(variable.name)

                # Asssuring that if the container is chosen, at least one of its cylinders is chosen.
                has_chosen_cylinder = False
                for box_from_container in containers[variable.name].boxes:
                    for cylinder_from_box in box_from_container.cylinders:
                        if cylinder_from_box.variable.varValue == 1:
                            has_chosen_cylinder = True
                            break
                if not has_chosen_cylinder:
                    return None

            # If the variable's name has at least one pipe symbol, then it is a cylinder.
            else:
                cylinder = cylinders[variable.name]
                chosen_cylinders_ids.append(cylinder.identifier)
                chosen_cylinders.append(cylinder)
                sum_volumes += cylinder.volume
                sum_weights += cylinder.weight
    
                # Boxes are identified by using the container and box identifiers concatenated.
                container_box_identifier = split_variable_name[0] + '|' + split_variable_name[1]
                # Assuring that if the cylinder is chosen, then the container of the selected cylinder is also chosen.
                if cylinder.box.container.variable.varValue != 1:
                    return None
                # Assuring that all boxes were used at most once.
                if container_box_identifier not in used_boxes:
                    used_boxes.add(container_box_identifier)
                else:
                    return None

    # Checking if this solution is unique. The solution is converted to one string and checked if it is already stored in all_optimal_solutions
    solution_string = ','.join(chosen_cylinders_ids)
    if solution_string in all_optimal_solutions:
        return None
    else:
        all_optimal_solutions.add(solution_string)

    # Rounding the variables to avoid numerical errors
    DIGITS_PRECISION = 2
    sum_volumes = round(sum_volumes, DIGITS_PRECISION)
    sum_weights = round(sum_weights, DIGITS_PRECISION)
    # Checking whether the other metrics are also correct.
    if len(used_containers) == TOTAL_CONTAINERS and sum_volumes == TOTAL_VOLUME and sum_weights == TOTAL_WEIGHT:
        return chosen_cylinders_ids, chosen_cylinders
    else:
        return None

Defining function for writing the optimal solutions into text files

In [47]:
folder_path = Path('./Solutions')
# Creates the folder 'Solutions' if it does not exist yet and stores one text file per solution.
def write_optimal_solution(optimal_solution, counter_solution):
    # Create the folder and also parent folders if they do not exist yet.
    folder_path.mkdir(parents=True, exist_ok=True)

    file_name = folder_path / (str(counter_solution) + '.txt')
    # Creating and writing the file.
    with open(file_name, 'w+') as file:
        file.write('\n'.join(cylinder for cylinder in optimal_solution))

Checking the optimal solution, looking for different optimal solutions, and storing the found solutions in text files

In [48]:
MAX_DIFFERENT_SOLUTIONS = 100
# Checking if the last different solution (including the original) was already generated. If so, avoiding generating all solutions.
last_file = folder_path / (str(MAX_DIFFERENT_SOLUTIONS + 1) + '.txt')

if not last_file.is_file():
    counter_different_solutions = 0
    # Checking the first found solution. The other iterations will check the possibly different solutions generated.
    while pulp_status_solve == LpStatusOptimal and counter_different_solutions <= MAX_DIFFERENT_SOLUTIONS:
        solution_return = evaluate_solution(problem)
        if solution_return is not None:
            optimal_solution, optimal_cylinder_objects = solution_return
            write_optimal_solution(optimal_solution, counter_different_solutions + 1)

            # Looking for other optimal solutions.
            # This additional constraint forces the model to find a solution without at least one of the chosen cylinders.
            problem += Item.sum_variables(optimal_cylinder_objects) <= len(optimal_cylinder_objects) - 1, "Generating different solution " + str(counter_different_solutions)
            pulp_status_solve = problem.solve(solver_object)
            counter_different_solutions += 1
        else:
            break

Reading a file and printing its solution

In [49]:
# Printing the whole row without breaking the line.
MAX_ROWS_PRINT = 100
MAX_COLUMNS_PRINT = 10
MAX_CHARACTERS_PRINT = MAX_ROWS_PRINT * MAX_COLUMNS_PRINT
pd.set_option('display.max_rows', MAX_ROWS_PRINT)
pd.set_option('display.max_columns', MAX_COLUMNS_PRINT)
pd.set_option('display.width', MAX_CHARACTERS_PRINT)

file_to_read = 1
file_name = 'Solutions/' + str(file_to_read) + '.txt'
if os.path.exists(file_name):
    file = open(file_name, 'r')
    # Read the file and converts it to a list of strings.
    # It is better than the readlines() function because it already removes the '\n' characters.
    optimal_solution = file.read().splitlines()
    print(data_frame.loc[optimal_solution])
else:
    print('File not found. Please select other solution.')

           Container   Box  Cylinder  Cylinder weight (g)  Cylinder volume (mL)  Density (g/mL)
identifier                                                                                     
AA|LB_1|1         AA  LB_1         1                  100                 15.72        6.361323
AA|LB_2|1         AA  LB_2         1                  196                 71.27        2.750105
AB|LB_1|1         AB  LB_1         1                  999                108.35        9.220120
AB|LB_2|1         AB  LB_2         1                  343                 78.67        4.359985
AC|LB_1|16        AC  LB_1        16                  141                 14.42        9.778086
AD|LB_1|1         AD  LB_1         1                   71                 35.50        2.000000
AD|LB_2|1         AD  LB_2         1                  176                115.79        1.519993
AF|LB_1|14        AF  LB_1        14                  321                226.06        1.419977
AG|LB_1|1         AG  LB_1         1    